<a href="https://colab.research.google.com/github/iamsourad/Fish-Disease-detection-using-Deep-Learning/blob/main/FISH_DISEASE_DETECTION_S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define paths to your dataset (adjust these paths based on your dataset location)
train_dir = '/content/drive/MyDrive/Dataset FISH/Freshwater Fish Disease/Train'
val_dir = '/content/drive/MyDrive/Dataset FISH/Freshwater Fish Disease/Valid'
test_dir = '/content/drive/MyDrive/Dataset FISH/Freshwater Fish Disease/Test'

In [ ]:
# Image parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
NUM_CLASSES =  7  # Example: Healthy, White Spot, Black Spot, Red Spot

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # EfficientNetB0-specific preprocessing
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
# Load and preprocess data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 1747 images belonging to 7 classes.
Found 350 images belonging to 7 classes.
Found 350 images belonging to 7 classes.


In [ ]:
# Build EfficientNetB0 model
def build_efficientnet_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model.trainable = False  # Freeze base model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Build and train the model
model = build_efficientnet_model()
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,442 (16.08 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    epochs=30,  # Consistent with MobileNetV2 for fair comparison
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 765s 13s/step - accuracy: 0.3196 - loss: 1.7312 - val_accuracy: 0.6543 - val_loss: 1.0385
Epoch 2/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 487ms/step - accuracy: 0.6817 - loss: 0.9653 - val_accuracy: 0.7943 - val_loss: 0.6760
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 487ms/step - accuracy: 0.7463 - loss: 0.7724 - val_accuracy: 0.8257 - val_loss: 0.5767
Epoch 4/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 486ms/step - accuracy: 0.8094 - loss: 0.6012 - val_accuracy: 0.8914 - val_loss: 0.4051
Epoch 5/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 485ms/step - accuracy: 0.8315 - loss: 0.5249 - val_accuracy: 0.9029 - val_loss: 0.3539
Epoch 6/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 485ms/step - accuracy: 0.8602 - loss: 0.4340 - val_accuracy: 0.9257 - val_loss: 0.2736
Epoch 7/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 483ms/step - accuracy: 0.8702 - loss: 0.3830 - val_accuracy: 0.9286 - val_loss: 0.2519
Epoch 8/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 484ms/step - accuracy: 0.8830 - loss: 0.3264 - val_accur

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 107s 11s/step - accuracy: 0.9947 - loss: 0.0247
Test Accuracy: 0.9914


In [ ]:
# Save the model
model.save('/content/drive/MyDrive/MODELS/fish_disease_model_efficientnet.keras')

In [ ]:
base_model = model.layers[0]  # EfficientNetB0 is the first layer

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:200]:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint_cb = ModelCheckpoint('/content/drive/MyDrive/MODELS/fish_disease_model_efficientnet.keras', monitor='val_accuracy', save_best_only=True)
earlystop_cb = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

In [ ]:
fine_tune_epochs = 20
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs,
    callbacks=[checkpoint_cb, earlystop_cb]
)

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 92s 983ms/step - accuracy: 0.8407 - loss: 0.4734 - val_accuracy: 0.9800 - val_loss: 0.0606
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 527ms/step - accuracy: 0.8701 - loss: 0.3948 - val_accuracy: 0.9771 - val_loss: 0.0727
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 28s 500ms/step - accuracy: 0.8815 - loss: 0.3482 - val_accuracy: 0.9771 - val_loss: 0.0794
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 518ms/step - accuracy: 0.9054 - loss: 0.2981 - val_accuracy: 0.9771 - val_loss: 0.0831
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 491ms/step - accuracy: 0.8901 - loss: 0.3046 - val_accuracy: 0.9771 - val_loss: 0.0827
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 497ms/step - accuracy: 0.9098 - loss: 0.2785 - val_accuracy: 0.9743 - val_loss: 0.0821
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 495ms/step - accuracy: 0.9123 - loss: 0.2695 - val_accuracy: 0.9743 - val_loss: 0.0804
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 493ms/step - accuracy: 0.9356 - loss: 0.2191 - val_accu

In [ ]:
# Evaluation
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(val_generator)
test_loss, test_acc = model.evaluate(test_generator)

55/55 ━━━━━━━━━━━━━━━━━━━━ 28s 518ms/step - accuracy: 0.9796 - loss: 0.0619
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.9918 - loss: 0.0451
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.9877 - loss: 0.0572


In [ ]:
print(f"Train Accuracy: {train_acc:.2f}")
print(f"Validation Accuracy: {val_acc:.2f}")
print(f"Test Accuracy: {test_acc:.2f}")

Train Accuracy: 0.98
Validation Accuracy: 0.98
Test Accuracy: 0.99


In [ ]:
# Save Fine-Tuned Model
model.save('/content/drive/MyDrive/MODELS/efficientnet_finetuned.keras')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/MODELS/efficientnet_finetuned.keras')

In [ ]:
# Evaluation
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(val_generator)
test_loss, test_acc = model.evaluate(test_generator)

55/55 ━━━━━━━━━━━━━━━━━━━━ 37s 517ms/step - accuracy: 0.9874 - loss: 0.0563
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 545ms/step - accuracy: 0.9790 - loss: 0.0607
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.9877 - loss: 0.0572


In [ ]:
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

Train Accuracy: 0.9880
Validation Accuracy: 0.9800
Test Accuracy: 0.9886


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np

# Define your class names
class_names = [
    'Bacterial Red Disease',
    'Bacterial Diseases (Aeromoniasis)',
    'Bacterial Gill Disease',
    'Fungal Diseases (Saprolegniasis)',
    'Parasitic Diseases',
    'Viral Diseases (White Tail Disease)',
    'Healthy Fish'
]

# Predict on test dataset
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes  # True labels
accuracy = accuracy_score(y_true, y_pred)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Add overall accuracy as a column
df_report["Accuracy"] = ""
df_report.loc["accuracy", "Accuracy"] = f"{accuracy * 100:.2f}%"

# Keep only necessary columns and rename
df_clean = df_report.loc[class_names][["precision", "recall", "f1-score"]]
df_clean.columns = ["Precision", "Recall", "F1-Score"]
df_clean["Accuracy"] = f"{accuracy * 100:.2f}%"  # Add accuracy to each row for table view

# Print or export the result
print(df_clean)

# Optional: Save to CSV or Excel
# df_clean.to_csv("classification_report.csv")
# df_clean.to_excel("classification_report.xlsx")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 232s 22s/step
                                     Precision  Recall  F1-Score Accuracy
Bacterial Red Disease                 1.000000    0.98  0.989899   98.86%
Bacterial Diseases (Aeromoniasis)     1.000000    1.00  1.000000   98.86%
Bacterial Gill Disease                1.000000    1.00  1.000000   98.86%
Fungal Diseases (Saprolegniasis)      0.960784    0.98  0.970297   98.86%
Parasitic Diseases                    1.000000    1.00  1.000000   98.86%
Viral Diseases (White Tail Disease)   1.000000    0.96  0.979592   98.86%
Healthy Fish                          0.961538    1.00  0.980392   98.86%


In [ ]:
df_clean.to_csv("/content/drive/MyDrive/REPORT_CAL(FISH)/classification_report.csv")
df_clean.to_excel("/content/drive/MyDrive/REPORT_CAL(FISH)/classification_report.xlsx")